# The Drazin Inverse Exercise

### Suleyman Gozen
 
 I thank Yung-Hsu Tsui for his valuable comments.

In [1]:
import numpy as np
import pandas as pd
from scipy import linalg as la
from scipy.sparse import csgraph

In [2]:
""" Problem 1 """

def drazin(A,k,AD):

    if np.allclose(A@AD, AD@A) and \
    np.allclose(np.linalg.matrix_power(A,k+1)@AD, np.linalg.matrix_power(A,k)) and \
    np.allclose(AD@A@AD,AD):
        return True
    else:
        return False

A = np.array([[1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]])
AD = np.array([[1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]])
Ak = 1

B = np.array([[1,1,3],[5,2,6],[-2,-1,-3]])
BD = np.zeros_like(B)
Bk = 3

print(drazin(A,Ak,AD))
print(drazin(B,Bk,BD))

True
True


In [3]:
""" Problem 2 """

def get_Drazin(A,tol):
    n,n = A.shape
    f = lambda x: abs(x) > tol
    g = lambda x: abs(x) <= tol
    Q1,S,k1 = la.schur(A, sort=f)
    Q2,T,k2 = la.schur(A, sort=g)
    U = np.concatenate((S[:,:k1],T[:,:n-k1]),axis = 1)
    UI = la.inv(U)
    V = UI@A@U
    Z = np.zeros((n,n))
    if k1 != 0:
        MI = la.inv(V[:k1,:k1])
        Z[:k1,:k1] = MI
    return U@Z@UI

AD = get_Drazin(A,1e-5)
print(AD)
print(drazin(A,1,AD))

BD = get_Drazin(B,1e-5)
print(BD)
print(drazin(B,3,BD))

[[  1.  -3.   9.  81.]
 [  0.   1.  -3. -18.]
 [  0.   0.   1.   3.]
 [  0.   0.   0.   0.]]
True
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
True


In [4]:
""" Problem 3 """

def eff_resistance(A):
    n,n = A.shape
    L = csgraph.laplacian(A)
    R = np.zeros((n,n))
    I = np.eye(n)
    for i in range(n):
        Lj = np.copy(L)
        Lj[i,:] = I[i,:] 
        Lj_drazin = get_Drazin(Lj, 1e-5)
        R[:,i] = np.diag(Lj_drazin)
        R[i,i] = 0
    return R


#Figure 16.2, Column 2 Test
A = np.array([[0,1],[1,0]])
print("\n",eff_resistance(A))

B = np.array([[0,3],[3,0]])
print("\n", eff_resistance(B))

C = np.array([[0,4],[4,0]])
print("\n",eff_resistance(C))


 [[ 0.  1.]
 [ 1.  0.]]

 [[ 0.          0.33333333]
 [ 0.33333333  0.        ]]

 [[ 0.    0.25]
 [ 0.25  0.  ]]


In [5]:
""" Problem 4 and 5 """

class LinkPredictor:
    
    def __init__(self, filename = "social_network.csv"):
    
        with open(filename, 'r') as file:
            lines = file.readlines()
            
        network = np.empty(shape=[0, 2])
        for line in lines:
            line = line.split(',')
            network = np.append(network, [[line[0], line[1].strip()]], axis=0)
        names = np.unique(network)
        index = {k: v for v, k in enumerate(names)}

        matrix = np.zeros((len(names),len(names)))
        for i in range(len(network[:,0])):
            matrix[index[network[i,0]],index[network[i,1]]] = 1
            matrix[index[network[i,1]],index[network[i,0]]] = 1

        self.names = names
        self.index = index
        self.A = matrix
        self.eff_R = eff_resistance(matrix)
 
    def predict_link(self, node=None):
        
        eff_R = self.eff_R.copy()
        eff_R[self.A != 0] = 0
        
        if node != None:
            if node not in self.names:
                return ValueError(node + " not in list of names")
            else:
                nodecol = eff_R[:, self.index[node]]
                minval = np.min(nodecol[nodecol != 0])
                loc = np.where(nodecol == minval)
                return self.names[loc[0]]
        else:
            minval = np.min(eff_R[eff_R != 0])
            loc = np.where(eff_R == minval)
            return self.names[loc[0][0]], self.names[loc[1][0]]
            
        
    def add_link(self, node1, node2):
        
        if node1 not in self.names:
            raise ValueError(node1 + " not in list of names")
        if node2 not in self.names:
            raise ValueError(node2 + " not in list of names")   

        self.A[self.index[node1],self.index[node2]] = 1
        self.A[self.index[node2],self.index[node1]] = 1
        self.eff_R = eff_resistance(self.A)

        
graph = LinkPredictor()
print(graph.predict_link())
print(graph.predict_link("Melanie"))
print(graph.predict_link("Alan"))
graph.add_link("Alan", "Sonia")
print(graph.predict_link("Alan"))
graph.add_link("Alan", "Piers")
print(graph.predict_link("Alan"))

('Oliver', 'Emily')
['Carol']
['Sonia']
['Piers']
['Abigail']
